In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from detector import Detector

In [2]:
# Get the path to the 'scans' directory using the current file's location
script_directory = os.getcwd()
scans_directory = os.path.join(script_directory, '..', 'data/scans')
output_directory = os.path.join(script_directory, '..', 'data/outputs')

# Specify the path to your .tif file
file = '7b'
tif_file_path = scans_directory + "/" + f"{file}.tif"
filename_4_saving = output_directory + "/" + f"{file}.png"

# Read the .tif file
image = cv2.imread(tif_file_path, cv2.IMREAD_UNCHANGED)
MIN_AREA = 2000
MAX_AREA = 1000000
 
scale_factor = {'x': 0.4, 'y': 0.4}
image = cv2.resize(image, None, fx=scale_factor['x'], fy=scale_factor['y'])
grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def show(image, name='Result'):
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:

# Provide morphological operations and find edges
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (4, 4), 0)
edged = cv2.Canny(blurred, 10, 100)

# Define a (3, 3) structuring element
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

# Apply the dilation operation to the edged image
dilate = cv2.dilate(edged, kernel, iterations=1)

error: OpenCV(4.6.0) /croot/opencv-suite_1691620365762/work/modules/imgproc/src/smooth.dispatch.cpp:293: error: (-215:Assertion failed) ksize.width > 0 && ksize.width % 2 == 1 && ksize.height > 0 && ksize.height % 2 == 1 in function 'createGaussianKernels'


In [4]:
# Contours detection
contours, hierarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
image_copy = image.copy()
borders = []
# Contours drawing
for cnt in contours:
    # Get the 4 points of the bounding rectangle
    x, y, w, h = cv2.boundingRect(cnt)
    # if MIN_AREA < w * h < MAX_AREA:
        # Draw a straight rectangle with the points
    cv2.rectangle(image_copy, (x, y), (x + w, y + h), color=(255, 0, 0), thickness=2)
    borders.append([x, y, w, h])

NameError: name 'dilate' is not defined

In [5]:
show(dilate)

In [6]:
# https://www.fypsolutions.com/opencv-python/findcontours-opencv-python-drawcontours-opencv-python/
gray = cv2.GaussianBlur(grayImage, (7, 7), 0) # 7
gray= cv2.medianBlur(gray, 3)   #to remove salt and paper noise
#to binary
ret,thresh = cv2.threshold(gray,190,255,0)  #to detect white objects
#to get outer boundery only
kernel = np.ones((2,2),np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_GRADIENT, kernel)
#to strength week pixels
thresh = cv2.dilate(thresh,kernel,iterations = 5)

In [28]:
def preprocessing(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    show(gray, 'gray')
    # blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(input=gray, edged=50, threshold1=100, threshold2=1, L2radient=True)
    show(edged, 'edged')

    # define a (3, 3) structuring element
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # apply the dilation operation to the edged image
    dilate = cv2.dilate(edged, kernel, iterations=2) # init 1
    show(dilate, 'dilate')
    return dilate

In [6]:
def find_contours(image, dilate):
    # contours detection
    contours,hierarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_4_plot = image.copy()
    areas = []
    crop_offset = 3
    for cnt in contours:
        # Get the 4 points of the bounding rectangle
        x, y, w, h = cv2.boundingRect(cnt)
        if w*h>20000:
            # Draw a straight rectangle with the points
            img_box = cv2.rectangle(image_4_plot, (x, y), (x+w, y+h), color = (255, 0, 0), thickness = 2)
            areas.append([x, y, w, h])
    show(image_4_plot, 'Contours')
    return image_4_plot, contours, areas

def crop(image, areas):
    mats = []
    crop_offset = 3
    for area in areas:
        # Cropping an image
        x, y, w, h = area
        cropped_image = image[y-crop_offset:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]
        mats.append(cropped_image)
        show(cropped_image, 'Crop')
    return mats
# cv2.imwrite(filename_4_saving, image_4_plot)

In [ ]:
for i, mat in enumerate(mats):
    cv2.imwrite(output_directory + "/" + f"{file}_{i}.png", mat)

In [29]:
dilate = preprocessing(image)
img, contours, areas = find_contours(image, dilate)
mats = crop(image, areas)

# show(mats[0], 'Mat')
# dilate1 = preprocessing(mats[0])
# img1, contours1, areas1 = find_contours(mats[0], dilate1)
# show(img1)


img_copy = image.copy()
crop_offset = 3
cropped_images = []
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > 100000:
        # Detect the convex contour
        hull = cv2.convexHull(cnt)
        img_hull = cv2.drawContours(img_copy, contours = [hull],
                                    contourIdx = 0,
                                    color = (255, 0, 0), thickness = 2)
        # Created a new mask and used bitwise_and to select for contours:
        mask = np.ones_like(img_copy[:, :, 0], dtype=np.uint8)
        cv2.drawContours(mask, [hull], -1, 0, -1)

        # Create a white image of the same size as the original image
        white_image = np.ones_like(image) * 255

        # Apply the mask: where mask is 0, use white_image; where mask is 1, use the original image
        masked_image = np.where(mask[:, :, np.newaxis] == 1, white_image, image)

        # Cropping an image
        x, y, w, h = cv2.boundingRect(cnt)
        cropped_image = masked_image[y-crop_offset:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]
        cropped_images.append(cropped_image)
        show(cropped_image)
# show(img_copy)

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'Canny'
> Overload resolution failed:
>  - Canny() missing required argument 'image' (pos 1)
>  - Canny() missing required argument 'image' (pos 1)
>  - Canny() missing required argument 'dx' (pos 1)
>  - Canny() missing required argument 'dx' (pos 1)


In [23]:
dilate1 = preprocessing(cropped_images[0])
img1, contours1, areas1 = find_contours(cropped_images[0], dilate1)
# show(img1)

img1_copy = img1.copy()

for cnt in contours1:
    area = cv2.contourArea(cnt)
    if area > 100000:
        # Detect the convex contour
        hull = cv2.convexHull(cnt)
        img_hull = cv2.drawContours(img1_copy, contours = [hull],
                                    contourIdx = 0,
                                    color = (255, 0, 0), thickness = 2)
        # Created a new mask and used bitwise_and to select for contours:
        # mask = np.ones_like(img1_copy[:, :, 0], dtype=np.uint8)
        # cv2.drawContours(mask, [hull], -1, 0, -1)
        #
        # # Create a white image of the same size as the original image
        # white_image = np.ones_like(img1) * 255
        #
        # # Apply the mask: where mask is 0, use white_image; where mask is 1, use the original image
        # masked_image = np.where(mask[:, :, np.newaxis] == 1, white_image, img1)
        #
        # # Cropping an image
        # x, y, w, h = cv2.boundingRect(cnt)
        # cropped_image = masked_image[y-crop_offset:(y+h+crop_offset), x-crop_offset:(x+w+crop_offset)]
        # cropped_images.append(cropped_image)
        show(img_hull)

In [ ]:
cv2.contorArea(contours)

In [26]:
# contours = max(contours, key = lambda x: cv2.contourArea(x))
# Created a new mask and used bitwise_and to select for contours:
hull = cv2.convexHull(contours)
mask = np.zeros_like(image)
cv2.drawContours(mask, [hull], -1, (255, 255, 255), -1)
masked_image = cv2.bitwise_and(image, mask)
show(masked_image)

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'convexHull'
> Overload resolution failed:
>  - points is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'points'


In [25]:
# ret , thrash = cv2.threshold(grayImage, 226 , 255, cv2.CHAIN_APPROX_NONE)
contours , hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
image_4_plot = image.copy()
for contour in contours:
    area = cv2.contourArea(contour)
    if area>150000:
        print(area)
        approx = cv2.approxPolyDP(contour, 0.01* cv2.arcLength(contour, True), True)
        cv2.drawContours(image_4_plot, [approx], 0, (255, 0, 0), 2)
        x = approx.ravel()[0]
        y = approx.ravel()[1] - 5

        if len(approx) == 3:
            cv2.putText(image_4_plot, "Triangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0) )
        elif len(approx) == 4 :
            x, y , w, h = cv2.boundingRect(approx)
            aspectRatio = float(w)/h
            print(aspectRatio)
            if aspectRatio >= 0.95 and aspectRatio < 1.05:
                cv2.putText(image_4_plot, "square", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

            else:
                cv2.putText(image_4_plot, "rectangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

        elif len(approx) == 5 :
            cv2.putText(image_4_plot, "pentagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
        elif len(approx) == 10 :
            cv2.putText(image_4_plot, "star", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
        else:
            cv2.putText(image_4_plot, "circle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

cv2.imshow('shapes', image_4_plot)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
def nothing(x):pass


cv2.namedWindow('videoUI', cv2.WINDOW_NORMAL)
cv2.createTrackbar('T','videoUI',0,255,nothing)

while(True):
    # vid_gray = cv2.cvtColor(grayImage, cv2.COLOR_BGR2GRAY)
    thresh = cv2.getTrackbarPos('T','videoUI')
    ret,thresh = cv2.threshold(gray,thresh,255,0)  #to detect white objects
    #to get outer boundery only
    kernel = np.ones((2,2),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_GRADIENT, kernel)
    #to strength week pixels
    thresh = cv2.dilate(thresh,kernel,iterations = 5)
    # vid_bw = cv2.threshold(grayImage, thresh, 255, cv2.THRESH_BINARY)[1]

    cv2.imshow('videoUI',cv2.flip(thresh,1))

    if cv2.waitKey(0) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
